In [4]:
#IN THIS CELL, EDIT METADATA & ADD MOUSE NAMES
from pynwb import TimeSeries
from pynwb.behavior import BehavioralEvents
from simply_nwb.transforms import labjack_load_file, mp4_read_data
from simply_nwb import SimpleNWB
from simply_nwb.transforms import plaintext_metadata_read
from dict_plus.utils.simpleflatten import SimpleFlattener
from simply_nwb.transforms import csv_load_dataframe_str
from pynwb.file import Subject
import pendulum
import numpy as np
import pandas as pd
import pickle
import os
import uuid
import glob
from pathlib import Path
from simply_nwb.util import panda_df_to_list_of_timeseries
from simply_nwb.pipeline import NWBSession
from simply_nwb.pipeline.enrichments.saccades import PutativeSaccadesEnrichment
from pynwb import NWBHDF5IO

# Simply-NWB Package Documentation
# https://simply-nwb.readthedocs.io/en/latest/index.html


SKIP = True  # if true, then we skip sessions that have already generated an NWB (for that particular date)

INSTITUTION = "CU Anschutz"
SESSION_DESCRIPTION = "change my value below, leave me here"
SESSIONS_TO_PROCESS = ["change my value below, leave me here"]
EXPERIMENTERS = [
    "Buteau, Anna"
]
LAB = "Felsen Lab"

EXPERIMENT_DESCRIPTION = "Evaluation of Perisaccadic Perceptual Changes in Mice"
EXPERIMENT_KEYWORDS = ["mouse", "saccades", "perception", "behavior", "licking"]
EXPERIMENT_RELATED_PUBLICATIONS = None

METADATA_FILENAME = "metadata.txt"

# Need multiple labjack datas?
LABJACK_FOLDER = "labjack/"
LABJACK_SUBFOLDER_GLOB = "*dread*"

LABJACK_NAME = "LabjackData"
LABJACK_SAMPLING_RATE = 1000.0  # in Hz
LABJACK_DESCRIPTION = "TTL signal for when the probe, frame and airpuff is present"
LABJACK_COMMENTS = "labjack data"

MP4_FILES = {
    "RightEye": "videos/*_rightCam-0000.mp4",
    "LeftEye": "videos/*_leftCam-0000.mp4"
}
MP4_DESCRIPTION = "Camera watching the eye and and tongue"
MP4_SAMPLING_RATE = 150.0

RESPONSE_SAMPLING_RATE = MP4_SAMPLING_RATE
RESPONSE_DESCRIPTION = "description about the processed response"
RESPONSE_COMMENTS = "comments about the response"

STIM_CSVS = {
    "LeftCamStim": {
        "csv_glob": "videos/*_leftCam*.csv",
        # Units line up with
        #         bodyparts,tongue,tongue,tongue,spout,spout,spout
        "units": ["idx", "px", "px", "likelihood", "px", "px", "likelihood"]
    },
    "RightCamStim": {
        "csv_glob": "videos/*_rightCam*.csv",
        # Units line up with
        #         bodyparts,center,center,center,nasal,nasal,nasal,temporal,temporal,temporal,dorsal,dorsal,dorsal,ventral,ventral,ventral
        "units": ["idx", "px", "px", "likelihood", "px", "px", "likelihood", "px", "px", "likelihood", "px", "px", "likelihood", "px", "px", "likelihood"]
    }
}

PROBE_SAMPLING_RATE = MP4_SAMPLING_RATE
PROBE_METADATA_FILE = "videos/driftingGratingWithProbeMetadata.txt"
PROBE_COMMENTS = "TODO comments about the probe here"
PROBE_DESCRIPTION = "TODO description of the probe data here"

SESSION_DESCRIPTION = "Air Puff2 - Static Grating Head-Fixed Paradigm"

MOUSE_DETAILS = {
    "lick1": {
        "birthday": pendulum.parse("10/19/21", strict=False),
        "sex": "M",
        "strain": "C57BL/6J"  # Wild
    },
    "dcm13": {
        "birthday": pendulum.parse("7/14/22", strict=False),
        "sex": "M",
        "strain": "Gad2-Cre"
    },
    "lick3": {
        "birthday": pendulum.parse("10/19/21", strict=False),
        "sex": "M",
        "strain": "C57BL/6J"  # Wild
    },
    "lick8": {
        "birthday": pendulum.parse("7/14/22", strict=False),
        "sex": "F",
        "strain": "Gad2-Cre"  # Wild
    },
    "DREADD11": {  # TODO unknown
        "birthday": pendulum.parse("1/1/2024", strict=False),
        "sex": "M",
        "strain": "Gad2-Cre"
    },
    "dreadd3.2": {
        "birthday": pendulum.parse("1/1/2024", strict=False),
        "sex": "M",
        "strain": "Gad2-Cre"
    },
    "dreadd4.2": {
        "birthday": pendulum.parse("1/1/2024", strict=False),
        "sex": "M",
        "strain": "Gad2-Cre"
    },
    "dreadd2.2": {
        "birthday": pendulum.parse("1/1/2024", strict=False),
        "sex": "M",
        "strain": "Gad2-Cre"
    }
}


def run_startup_checks(session_path):
    print("Checking mp4 files..")
    for mp4_name, mp4_glob in MP4_FILES.items():
        mp4_file_glob = os.path.join(session_path, mp4_glob)
        files = glob.glob(mp4_file_glob)
        if not files:
            raise ValueError(f"Couldn't find file with glob '{mp4_file_glob}'")


def create_nwb_file(session_path, MOUSE_DETAILS):
    mouse_name = os.path.basename(session_path)  # something like 'lick1' etc
    start_date = pendulum.now()

    if mouse_name not in MOUSE_DETAILS:
        raise ValueError(f"Unknown mouse '{mouse_name}', not found in MOUSE_DETAILS dict")

    birthday_diff = pendulum.now().diff(MOUSE_DETAILS[mouse_name]["birthday"])

    return start_date, SimpleNWB.create_nwb(
        session_description=SESSION_DESCRIPTION,
        session_start_time=start_date,
        experimenter=EXPERIMENTERS,
        subject=Subject(**{
            "subject_id": mouse_name,
            "age": f"P{birthday_diff.days}D",  # ISO-8601 for 90 days duration
            "strain": MOUSE_DETAILS[mouse_name]["strain"],
            "description": f"Mouse id '{mouse_name}'",
            "sex": MOUSE_DETAILS[mouse_name]["sex"]
        }),
        lab=LAB,
        experiment_description=EXPERIMENT_DESCRIPTION,
        session_id=mouse_name,
        institution=INSTITUTION,
        keywords=EXPERIMENT_KEYWORDS,
        related_publications=EXPERIMENT_RELATED_PUBLICATIONS
    )


def process_pickle_data(nwbfile, session_path):
    pickle_file_obj = open(os.path.join(session_path, PICKLE_FILENAME), "rb")
    pickle_data = pickle.load(pickle_file_obj)

    # Flatten the data so it's easier to add as a TimeSeries
    # e.g. {"data": {"a": [1,2,3]} }
    # gets transformed into
    # {"data_a": [1,2,3,4]}
    pickle_data = SimpleFlattener(simple_types=[np.ndarray, type(None)]).flatten(pickle_data)

    # Pop off all the None values from the pickle data
    keys_to_pop = []
    for k, v in pickle_data.items():
        if v is None:
            keys_to_pop.append(k)
    for k in keys_to_pop:
        pickle_data.pop(k)

    # Add all the data into timeseries
    timeseries_list = []
    for key, value in pickle_data.items():
        timeseries_list.append(
            TimeSeries(
                name=f"{key}",
                data=value,
                unit=PICKLE_DATA_UNITS[key],
                starting_time=0.0,
                rate=10.0,
                description=f"Measured {key}",
            ))

    # Add data to a NWBfile behavior module
    SimpleNWB.add_to_processing_module(
        nwbfile,
        module_name="behavior",
        data=BehavioralEvents(
            time_series=timeseries_list,
            name=f"BehavioralEvents"
        )
    )

    pickle_file_obj.close()


def process_labjack_data(nwbfile, session_path):
    labjack_folder = os.path.join(session_path, LABJACK_FOLDER)
    labjack_files = glob.glob(os.path.join(labjack_folder, "*.dat"))
    labjack_datas = []
    for labjack_file in labjack_files:
        filename = os.path.join(labjack_folder, labjack_file)
        labjack_datas.append(labjack_load_file(filename)["data"])

    labjack_combined = pd.concat(labjack_datas)

    timeseries_list = panda_df_to_list_of_timeseries(
        pd_df=labjack_combined,
        measured_unit_list=["s", "s", "s", "s", "s", "barcode", "s", "s", "s"],
        start_time=0.0,
        sampling_rate=LABJACK_SAMPLING_RATE,
        description=LABJACK_DESCRIPTION,
        comments=LABJACK_COMMENTS
    )

    SimpleNWB.add_to_processing_module(nwbfile, module_name="behavior", data=BehavioralEvents(
        time_series=timeseries_list,
        name=f"labjack_behavioral_events"
    ))


def process_mp4_data(nwbfile, session_path):
    # Add mp4 data to NWB
    for mp4_name, mp4_glob in MP4_FILES.items():
        print(f"Processing '{mp4_name}'..")
        mp4_file_glob = os.path.join(session_path, mp4_glob)
        files = glob.glob(mp4_file_glob)
        if not files:
            raise ValueError(f"Couldn't find file with glob '{mp4_file_glob}'")

        data, frames = mp4_read_data(files[0])

        SimpleNWB.mp4_add_as_acquisition(
            nwbfile,
            name=mp4_name,
            numpy_data=data,
            frame_count=frames,
            sampling_rate=MP4_SAMPLING_RATE,
            description=MP4_DESCRIPTION
        )


def putative_enrich(session_path, nwbfile):
    vid_folder = os.path.join(session_path)
    csvs = glob.glob(f"{vid_folder}/**rightCam*.csv", recursive=True)
    txts = glob.glob(f"{vid_folder}/**rightCam*timestamps.txt")
    assert len(csvs) > 0
    assert len(txts) > 0
    csv = csvs[0]
    txt = txts[0]
    
    enrich = PutativeSaccadesEnrichment.from_raw(
        nwbfile, csv, txt, 
        units=["idx", "px", "px", "likelihood", "px", "px", "likelihood", "px", "px", "likelihood", "px", "px", "likelihood", "px", "px", "likelihood"],
        x_center="center_x", y_center="center_y", likelihood="center_likelihood"
    )
    return enrich


def process_session(session_path, MOUSE_DETAILS):
    # /media/retina2/Seagate Portable Drive/NPData/2024-06-18/dreadd4.2
    # session_path
    print(f"Starting session processing of '{session_path}'..")
    mouse_name = os.path.basename(session_path)  # something like 'lick1' etc

    sesspath_obj = Path(session_path)
    folderdate = pendulum.parse(sesspath_obj.parent.name, strict=False)

    print("Running startup checks..")
    run_startup_checks(session_path)

    print("Creating NWB file..")
    start_date, nwbfile = create_nwb_file(session_path, MOUSE_DETAILS)

    
    nwbfilename = f"putative-nwb-{folderdate.day}-{folderdate.month}-{folderdate.year}-{mouse_name}-start-{start_date.month}-{start_date.day}.nwb"
    if os.path.exists(os.path.join("putative", nwbfilename)) and SKIP:
        print(f"File '{nwbfilename}' exists, skipping..")
        return
    
    # print("Reading labjack datas..")
    # process_labjack_data(nwbfile, session_path)
    # nwbfile.processing["behavior"]["labjack_behavioral_events"]["v0"].data

    # print("Reading response data..")
    # process_response_data(nwbfile, session_path)
    # nwbfile.processing["LeftCamStim"]["tongue_x"].data

    # print("Adding MP4 Data, might take a while..")
    # process_mp4_data(nwbfile, session_path)
    # nwbfile.acquistion["LeftEyeCam"]

    print("Enriching NWB with Putative Saccade Data")
    
    print("Enriching with putative NWB..")
    tmp_dir = "tmp"
    rand_fn = str(uuid.uuid4())
    if not os.path.exists(tmp_dir):
        os.mkdir(tmp_dir)
        
    print("Creating tmp NWB..")
    SimpleNWB.write(nwbfile, os.path.join(tmp_dir, f"{rand_fn}.nwb"))
    del nwbfile
    
    fp = NWBHDF5IO(os.path.join(tmp_dir, f"{rand_fn}.nwb"), "r")
    nwbfile = fp.read()

    rand_fn2 = str(uuid.uuid4())
    enrich = putative_enrich(session_path, nwbfile)
    SimpleNWB.write(nwbfile, os.path.join(tmp_dir, f"{rand_fn2}.nwb"))
    del nwbfile
       
    print("Writing NWB file, might take a while..")
    if not os.path.exists("putative"):
        os.mkdir("putative")
    
    sess = NWBSession(os.path.join(tmp_dir, f"{rand_fn2}.nwb"))
    sess.enrich(enrich)
    print("Writing NWB '{}'..".format(nwbfilename))
    sess.save(os.path.join("", nwbfilename))
    del sess
    
    print("Done!")
    return nwbfilename


In [5]:
#EDIT DATES OF SESSIONS TO PROCESS
import pendulum


# Scan entire directory code - uncomment
dirprefix = "/media/retina2/Seagate Portable Drive/NPData"
SESSIONS_TO_PROCESS = []
# for datename in os.listdir(dirprefix):
#     datedir = os.path.join(dirprefix, datename)
#     if os.path.isdir(datedir):
#         for mousename in os.listdir(datedir):
#             SESSIONS_TO_PROCESS.append(os.path.join(datedir, mousename))

month_gte = 6  # date greater than or equal to
day_gte = 18
for datename in os.listdir(dirprefix):
    datedir = os.path.join(dirprefix, datename)
    if os.path.isdir(datedir):
        split = datedir.split("-")
        month = split[1]
        day = split[2]
        if int(month) >= month_gte and int(day) >= day_gte:
            for mousename in os.listdir(datedir):
                SESSIONS_TO_PROCESS.append(os.path.join(datedir, mousename))



In [ ]:
SESSIONS_TO_PROCESS

In [6]:
filenames = []
for session in SESSIONS_TO_PROCESS:
    try:
        filenames.append(process_session(session, MOUSE_DETAILS))
    except Exception as e:
        print(f"Error with session '{session}'! Error '{str(e)}' Skipping..")
        continue


Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-18/dreadd3.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
File 'putative-nwb-18-6-2024-dreadd3.2-start-6-18.nwb' exists, skipping..
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-18/dreadd4.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
File 'putative-nwb-18-6-2024-dreadd4.2-start-6-18.nwb' exists, skipping..
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-19/dreadd3.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
File 'putative-nwb-19-6-2024-dreadd3.2-start-6-19.nwb' exists, skipping..
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-19/dreadd4.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
File 'putative

INFO:PutativeSaccades:Extracting eye position..
INFO:PutativeSaccades:Correcting eye position..
INFO:PutativeSaccades:Interpolating eye position..
INFO:PutativeSaccades:Decomposing eye position..
INFO:PutativeSaccades:Reorienting eye position..


Error with session '/media/retina2/Seagate Portable Drive/NPData/2024-06-21/dreadd4.2'! Error 'Could not determine correlation between raw and decomposed eye position. Column '0' corr_coeff '-0.036687263932404665' p_val '1.4257493080676049e-263'' Skipping..
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-22/dreadd2.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
File 'putative-nwb-22-6-2024-dreadd2.2-start-6-22.nwb' exists, skipping..
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-22/dreadd4.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
File 'putative-nwb-22-6-2024-dreadd4.2-start-6-22.nwb' exists, skipping..
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-23/dreadd2.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
File 'putative-nwb-23

INFO:PutativeSaccades:Extracting eye position..
INFO:PutativeSaccades:Correcting eye position..
INFO:PutativeSaccades:Interpolating eye position..
INFO:PutativeSaccades:Decomposing eye position..
INFO:PutativeSaccades:Reorienting eye position..
INFO:PutativeSaccades:Filtering eye position..
INFO:PutativeSaccades:Extracting putative saccades..
INFO:PutativeSaccades:Detected '2988' putative saccade waveforms under 'RightCamStim'
INFO:PutativeSaccades:Saving to NWB..
INFO:PutativeSaccades:Done


Writing NWB 'putative-nwb-24-6-2024-dreadd4.2-start-6-24.nwb'..
Done!
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-25/dreadd2.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
Enriching NWB with Putative Saccade Data
Enriching with putative NWB..
Creating tmp NWB..
Writing NWB file, might take a while..


...

Writing NWB 'putative-nwb-27-6-2024-dreadd2.2-start-6-27.nwb'..
Done!
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-27/dreadd4.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
Enriching NWB with Putative Saccade Data
Enriching with putative NWB..
Creating tmp NWB..
Writing NWB file, might take a while..


INFO:PutativeSaccades:Extracting eye position..
INFO:PutativeSaccades:Correcting eye position..
INFO:PutativeSaccades:Interpolating eye position..
INFO:PutativeSaccades:Decomposing eye position..
INFO:PutativeSaccades:Reorienting eye position..
INFO:PutativeSaccades:Filtering eye position..
INFO:PutativeSaccades:Extracting putative saccades..
INFO:PutativeSaccades:Detected '3367' putative saccade waveforms under 'RightCamStim'
INFO:PutativeSaccades:Saving to NWB..
INFO:PutativeSaccades:Done


Writing NWB 'putative-nwb-27-6-2024-dreadd4.2-start-6-27.nwb'..
Done!
Starting session processing of '/media/retina2/Seagate Portable Drive/NPData/2024-06-28/dreadd4.2'..
Running startup checks..
Checking mp4 files..
Creating NWB file..
Reading metadata file..
Enriching NWB with Putative Saccade Data
Enriching with putative NWB..
Creating tmp NWB..
Writing NWB file, might take a while..


INFO:PutativeSaccades:Extracting eye position..
INFO:PutativeSaccades:Correcting eye position..
INFO:PutativeSaccades:Interpolating eye position..
INFO:PutativeSaccades:Decomposing eye position..
INFO:PutativeSaccades:Reorienting eye position..
INFO:PutativeSaccades:Filtering eye position..
INFO:PutativeSaccades:Extracting putative saccades..
INFO:PutativeSaccades:Detected '2946' putative saccade waveforms under 'RightCamStim'
INFO:PutativeSaccades:Saving to NWB..
INFO:PutativeSaccades:Done


Writing NWB 'putative-nwb-28-6-2024-dreadd4.2-start-6-28.nwb'..
Done!


In [2]:
# GO TO VS CODE FOR TRAIN_PREDICTOIVE_NWB.PY
import glob
import os
import random

import pendulum
from pynwb import NWBHDF5IO
from pynwb.file import Subject
from simply_nwb import SimpleNWB
from simply_nwb.pipeline import NWBSession
from simply_nwb.pipeline.enrichments.saccades import PutativeSaccadesEnrichment
from simply_nwb.pipeline.enrichments.saccades.predict_gui import PredictedSaccadeGUIEnrichment

def select_putative_training_nwbs(list_of_nwbs, doskip):
    if doskip:
        return [list_of_nwbs[0]]
    num = len(list_of_nwbs)
    random.shuffle(list_of_nwbs)
    return list_of_nwbs[:5]

# Get the filenames for the timestamps.txt and dlc CSV
input_folder = "putative/"
output_folder = "predicted/"
skip_load_trainingdata = False

files = glob.glob(os.path.join(input_folder, "**.nwb"))
print("Creating enrichment..")
enrich = PredictedSaccadeGUIEnrichment(200, select_putative_training_nwbs(files, skip_load_trainingdata), 40, 
{"x_center": "center_x", 
 "y_center": "center_y", 
 "likelihood": "center_likelihood"
})

for file in files:
    savefn = os.path.join(output_folder, f"predictive-{os.path.basename(file)[:-len('.nwb')]}.nwb")
    if os.path.exists(savefn):
        print(f"File exists, skipping '{savefn}'..")
    print(f"Loading '{file}'..")
    sess = NWBSession(file)
    # Take our putative saccades and do the actual prediction for the start, end time, and time location
    print("Enriching..")
    sess.enrich(enrich)
    print("Saving to NWB")
    print(f"Saving to file {savefn}..")
    sess.save(savefn)  # Save as our finalized session, ready for analysis
    tw = 2


Creating enrichment..
Loading 'putative/putative-nwb-28-6-2024-dreadd4.2-start-6-28.nwb'..
Enriching..
MAKE SURE YOU HAVE AT LEAST 10 EXAMPLES OF LEFT AND RIGHT SACCADES!!
MAKE SURE YOU LABEL AT LEAST 5 EPOCHS!!


In [ ]:
# IGNORE BELOW!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
nwb

In [19]:
nwb.processing

{'behavior': behavior pynwb.base.ProcessingModule at 0x140120009564416
 Fields:
   data_interfaces: {
     BehavioralEvents <class 'pynwb.behavior.BehavioralEvents'>,
     labjack_behavioral_events <class 'pynwb.behavior.BehavioralEvents'>
   }
   description: behavior processing module}

In [17]:
nwb.processing["behavior"]

behavior pynwb.base.ProcessingModule at 0x140120009564416
Fields:
  data_interfaces: {
    BehavioralEvents <class 'pynwb.behavior.BehavioralEvents'>,
    labjack_behavioral_events <class 'pynwb.behavior.BehavioralEvents'>
  }
  description: behavior processing module

In [20]:
nwb.processing["behavior"]["labjack_behavioral_events"]

labjack_behavioral_events pynwb.behavior.BehavioralEvents at 0x140120009567248
Fields:
  time_series: {
    Time <class 'pynwb.base.TimeSeries'>,
    v0 <class 'pynwb.base.TimeSeries'>,
    v1 <class 'pynwb.base.TimeSeries'>,
    v2 <class 'pynwb.base.TimeSeries'>,
    v3 <class 'pynwb.base.TimeSeries'>,
    y0 <class 'pynwb.base.TimeSeries'>,
    y1 <class 'pynwb.base.TimeSeries'>,
    y2 <class 'pynwb.base.TimeSeries'>,
    y3 <class 'pynwb.base.TimeSeries'>
  }

In [21]:
nwb.processing["behavior"]["labjack_behavioral_events"]["v0"]

v0 pynwb.base.TimeSeries at 0x140120009566576
Fields:
  comments: labjack data
  conversion: 1.0
  data: <HDF5 dataset "data": shape (11932000,), type "<f8">
  description: column v0 TTL signal for when the probe, frame and airpuff is present
  offset: 0.0
  rate: 1000.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: s

In [28]:
# Actually get numpy data
nwb.processing["behavior"]["labjack_behavioral_events"]["v0"].data[:]

array([65408., 65408., 65408., ..., 65472., 65472., 65472.])

In [34]:
left_nasal = nwb.processing["behavior"]["BehavioralEvents"]["saccadeClassificationResults_left_nasal_indices"].data[:]
left_temporal = nwb.processing["behavior"]["BehavioralEvents"]["saccadeClassificationResults_left_temporal_indices"].data[:]

In [35]:
left_nasal

array([  3627,   4107,   6966,   7310,   7732,   8092,   8375,   9500,
         9747,   9999,  10057,  12829,  13257,  13601,  15535,  17212,
        44267,  47341,  50323,  50586,  50799,  51439,  51766,  58773,
        70159,  72006,  72481,  73043,  77117,  77734,  77901,  77972,
        78239,  80591,  84324,  85415,  87054,  91938,  94658, 100129,
       100692, 107523, 108431, 108780, 115341, 115415, 116462, 118569,
       124671, 127365, 136210, 138287, 138533, 139022, 139156, 145743,
       148291, 148703, 150437, 153381, 153498, 154946, 155062, 155479,
       155549, 156511, 157266, 161746, 166844, 169604, 171530, 172561,
       175204, 175786, 177907, 178340, 179212, 179860, 180205, 180504,
       180607, 180694, 180956, 181042, 181210, 181452, 182543, 182806,
       182889, 183386, 183509, 183587, 183890, 183988, 184316, 184865,
       184896, 184998, 185145, 185260, 185667, 185774, 187476, 187583,
       187911, 187986, 188346, 188570, 188653, 188717, 188827, 188899,
      